In [18]:
import json
import http.client
import boto3
import urllib3
import requests


s3_client = boto3.client('s3')

API_KEY = "secret"
API_HOST = "lg1uksflod.execute-api.us-east-1.amazonaws.com"
LOCAL_FILE_PATH = "/home/ec2-user/SageMaker"

def getpresignedURL(fileName):
    conn = http.client.HTTPSConnection(API_HOST)
    payload = ''
    headers = {
      'X-API-key': API_KEY,
      'Content-type': 'application/json'
    }
    conn.request("GET", "/prod/TechJam/presignedUrl/{}".format(fileName), payload, headers)
    res = conn.getresponse()
    return json.loads(res.read().decode('unicode_escape'))
    
def getRecommendation(fileName):
    conn = http.client.HTTPSConnection(API_HOST)
    payload = ''
    headers = {
      'X-API-key': API_KEY,
      'Content-type': 'application/json'
    }
    conn.request("GET", "/prod/TechJam/recommendation/{}".format(fileName), payload, headers)
    res = conn.getresponse()
    return json.loads(res.read().decode('unicode_escape'))["fileName"]
    
# def uploadFile(localFilePath, presignedUrl):
#     # Extract the hostname and the path from the presigned URL
#     url_parts = presignedUrl.replace('https://', '').split('/', 1)
#     host = url_parts[0]
#     path = '/' + url_parts[1]
#     with open(localFilePath, 'rb') as file:  
#         try:
#             print("file:{}".format(file))
#             s3conn = http.client.HTTPSConnection(host)
#             foo = {'text': 'Hello HTTP #1 **cool**, and #1!'}
#             json_data = json.dumps(foo)
#             s3conn.request("PUT", path, file)
#             return s3conn.getresponse()
#         except Exception as e:
#             raise e

def upload_file_to_s3_presigned_url(file_path, presigned_url):
    with open(file_path, 'rb') as file:
        try:
            response = requests.put(presigned_url, data=file)
            response.raise_for_status()  # Raises stored HTTPError, if one occurred.
            print(f"Upload Successful: {response.status_code}")
            return response
        except requests.exceptions.RequestException as e:
            print(f"Upload Failed: {e}")



In [19]:
    fileName = "hodu123.MOV"

    presignedUrl = getpresignedURL(fileName)
    
#     uploadSuccess = uploadFile(LOCAL_FILE_PATH, presignedUrl)

    localFilePath = "{}/{}".format(LOCAL_FILE_PATH, fileName)
    uploadSuccess = upload_file_to_s3_presigned_url(localFilePath, presignedUrl)
    
    recommendation = ""
    if(uploadSuccess.status_code == 200):
        recommendation = getRecommendation(fileName)
    else:
        raise Exception("uploading file Fail:{}".format(uploadSuccess.reason))
    
    print(recommendation)
        

Upload Successful: 200
hodu123.MOV


In [5]:
import os
os.path.abspath("")

'/home/ec2-user/SageMaker'